In [10]:
import gymnasium as gym
import time
import numpy as np

from stable_baselines3 import PPO, SAC, TD3
from stable_baselines3.common.vec_env import SubprocVecEnv
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.noise import NormalActionNoise


In [11]:
def make_env(rank, seed=0):
    def _init():
        env = gym.make("BipedalWalker-v3")
        env.reset(seed=seed + rank)
        env = Monitor(env)
        return env
    return _init


In [ ]:
N_ENVS_PPO = 4

env_ppo = SubprocVecEnv([make_env(i) for i in range(N_ENVS_PPO)])


In [13]:
ppo = PPO(
    "MlpPolicy",
    env_ppo,
    verbose=1,
    n_steps=512,      
    batch_size=64,
    gamma=0.99,
    learning_rate=3e-4,
)


Using cpu device


In [14]:
start = time.time()
ppo.learn(total_timesteps=1_000_000)
ppo_time = time.time() - start

ppo.save("ppo_bipedal_multi")
print(f"PPO entraîné en {ppo_time/60:.1f} minutes")


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 76.7     |
|    ep_rew_mean     | -107     |
| time/              |          |
|    fps             | 2936     |
|    iterations      | 1        |
|    time_elapsed    | 0        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 73.8        |
|    ep_rew_mean          | -107        |
| time/                   |             |
|    fps                  | 1592        |
|    iterations           | 2           |
|    time_elapsed         | 2           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008620907 |
|    clip_fraction        | 0.0731      |
|    clip_range           | 0.2         |
|    entropy_loss         | -5.68       |
|    explained_variance   | -0.000551   |
|    learning_rate        | 0.

In [15]:
N_ENVS_SAC = 2  # recommandé (off-policy)

env_sac = SubprocVecEnv([make_env(i) for i in range(N_ENVS_SAC)])


In [16]:
sac = SAC(
    "MlpPolicy",
    env_sac,
    verbose=1,
    buffer_size=1_000_000,
    batch_size=256,
    gamma=0.99,
    learning_rate=3e-4,
    ent_coef="auto",
)


Using cpu device


In [17]:
start = time.time()
sac.learn(total_timesteps=1_000_000)
sac_time = time.time() - start

sac.save("sac_bipedal_multi")
print(f"SAC entraîné en {sac_time/60:.1f} minutes")


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 838      |
|    ep_rew_mean     | -94.7    |
| time/              |          |
|    episodes        | 4        |
|    fps             | 156      |
|    time_elapsed    | 21       |
|    total_timesteps | 3368     |
| train/             |          |
|    actor_loss      | -17      |
|    critic_loss     | 0.0946   |
|    ent_coef        | 0.613    |
|    ent_coef_loss   | -3.28    |
|    learning_rate   | 0.0003   |
|    n_updates       | 1633     |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 862      |
|    ep_rew_mean     | -102     |
| time/              |          |
|    episodes        | 8        |
|    fps             | 146      |
|    time_elapsed    | 47       |
|    total_timesteps | 6988     |
| train/             |          |
|    actor_loss      | -23.8    |
|    critic_loss     | 0.253    |
|    ent_coef 

In [18]:
N_ENVS_TD3 = 2

env_td3 = SubprocVecEnv([make_env(i) for i in range(N_ENVS_TD3)])


In [19]:
n_actions = env_td3.action_space.shape[-1]

action_noise = NormalActionNoise(
    mean=np.zeros(n_actions),
    sigma=0.1 * np.ones(n_actions)
)

td3 = TD3(
    "MlpPolicy",
    env_td3,
    action_noise=action_noise,
    verbose=1,
    buffer_size=1_000_000,
    batch_size=256,
    gamma=0.99,
    learning_rate=3e-4,
)


Using cpu device


In [20]:
start = time.time()
td3.learn(total_timesteps=1_000_000)
td3_time = time.time() - start

td3.save("td3_bipedal_multi")
print(f"TD3 entraîné en {td3_time/60:.1f} minutes")


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 56.5     |
|    ep_rew_mean     | -104     |
| time/              |          |
|    episodes        | 4        |
|    fps             | 272      |
|    time_elapsed    | 0        |
|    total_timesteps | 240      |
| train/             |          |
|    actor_loss      | 0.822    |
|    critic_loss     | 217      |
|    learning_rate   | 0.0003   |
|    n_updates       | 69       |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 105      |
|    ep_rew_mean     | -116     |
| time/              |          |
|    episodes        | 8        |
|    fps             | 201      |
|    time_elapsed    | 4        |
|    total_timesteps | 844      |
| train/             |          |
|    actor_loss      | -0.12    |
|    critic_loss     | 111      |
|    learning_rate   | 0.0003   |
|    n_updates       | 371      |
--------------

In [21]:
eval_env = gym.make("BipedalWalker-v3")
eval_env = Monitor(eval_env)

ppo_mean, _ = evaluate_policy(ppo, eval_env, n_eval_episodes=10)
sac_mean, _ = evaluate_policy(sac, eval_env, n_eval_episodes=10)
td3_mean, _ = evaluate_policy(td3, eval_env, n_eval_episodes=10)

ppo_mean, sac_mean, td3_mean


(np.float64(276.7614373), np.float64(293.906714), np.float64(285.5740078))

In [22]:
import pandas as pd

results = pd.DataFrame({
    "Algorithme": ["PPO", "SAC", "TD3"],
    "Score moyen": [ppo_mean, sac_mean, td3_mean],
    "Temps entraînement (min)": [
        ppo_time/60,
        sac_time/60,
        td3_time/60
    ],
    "Multi-env": ["Oui (4)", "Oui (2)", "Oui (2)"]
})

results


,Algorithme,Score moyen,Temps entraînement (min),Multi-env
0,PPO,276.761437,13.838509,Oui (4)
1,SAC,293.906714,133.740442,Oui (2)
2,TD3,285.574008,116.193804,Oui (2)


In [ ]:
env_ppo.close()
env_sac.close()
env_td3.close()



In [24]:
import gymnasium as gym
import time

def render_model(model, name, seed=42, max_steps=2000):
    print(f"\n=== Affichage {name} ===")

    env = gym.make("BipedalWalker-v3", render_mode="human")
    obs, _ = env.reset(seed=seed)

    total_reward = 0

    for step in range(max_steps):
        action, _ = model.predict(obs, deterministic=True)
        obs, reward, terminated, truncated, _ = env.step(action)
        total_reward += reward

        if terminated or truncated:
            print(f"{name} terminé à l'étape {step}")
            break

        time.sleep(0.01)  # ralentit légèrement pour bien voir

    env.close()
    print(f"{name} — Reward cumulée : {total_reward:.2f}")


In [25]:
from stable_baselines3 import PPO, SAC, TD3

ppo = PPO.load("ppo_bipedal_multi")
sac = SAC.load("sac_bipedal_multi")
td3 = TD3.load("td3_bipedal_multi")


In [ ]:
render_model(ppo, "PPO", seed=42)
print("test")


=== Affichage PPO ===
PPO terminé à l'étape 1161
PPO — Reward cumulée : 292.58


In [30]:
render_model(sac, "SAC", seed=42)



=== Affichage SAC ===
SAC terminé à l'étape 768
SAC — Reward cumulée : 316.29


In [31]:
render_model(td3, "TD3", seed=42)



=== Affichage TD3 ===
TD3 terminé à l'étape 784
TD3 — Reward cumulée : 309.38
